In [13]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('HPCG.db')

In [14]:
def extract(filename, data):
    with open(filename, 'r') as file:
        lines = file.readlines()

    NX = ""
    NY = ""
    NZ = ""
    Time = ""
    # Although the time has been set to 1860, it varies
    Gflops = ""

    idd_index = 10
    # the line index of "Processor Dimensions="
    T_index = 89
    # the line index of "Benchmark Time Summary::Total=" 
    Gflops_index = 117
    # the line index of "Final Summary="

    NX = lines[idd_index + 1].split("=")[1]
    NY = lines[idd_index + 2].split("=")[1]
    NZ = lines[idd_index + 3].split("=")[1]
    Time = lines[T_index].split("=")[1]
    Gflops = lines[Gflops_index + 1].split("=")[1]

    data.append([NX, NY, NZ, Time, Gflops])
    return data        

In [15]:
import os

# check below before running this script
directory_path = "/home/lizz_lab/cse12111448/HPCG/hpcg/build/bin/"  
core_num = 24

# /home/lizz_lab/cse12111448/HPCG/hpcg/build/bin/HPCG-Benchmark_3.1_2023-11-13_15-36-13.txt
file_names = [file for file in os.listdir(directory_path) if file.startswith("HPCG-Benchmark") and file.endswith(".txt")]
data = []

for filename in file_names:
    print(filename)
    extract(directory_path + filename, data)

# conversion
data = [
    [int(value) if index in range(3) else float(value) if index == 4 else value for index, value in enumerate(row)]
    for row in data
]


# 加核数
data = [[core_num] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'NX', 'NY', 'NZ', 'Time', 'Gflops'])
df

HPCG-Benchmark_3.1_2023-11-11_15-55-20.txt
['HPCG-Benchmark\n', 'version=3.1\n', 'Release date=March 28, 2019\n', 'Machine Summary=\n', 'Machine Summary::Distributed Processes=24\n', 'Machine Summary::Threads per processes=1\n', 'Global Problem Dimensions=\n', 'Global Problem Dimensions::Global nx=208\n', 'Global Problem Dimensions::Global ny=416\n', 'Global Problem Dimensions::Global nz=312\n', 'Processor Dimensions=\n', 'Processor Dimensions::npx=2\n', 'Processor Dimensions::npy=4\n', 'Processor Dimensions::npz=3\n', 'Local Domain Dimensions=\n', 'Local Domain Dimensions::nx=104\n', 'Local Domain Dimensions::ny=104\n', 'Local Domain Dimensions::Lower ipz=0\n', 'Local Domain Dimensions::Upper ipz=2\n', 'Local Domain Dimensions::nz=104\n', '########## Problem Summary  ##########=\n', 'Setup Information=\n', 'Setup Information::Setup Time=21.775\n', 'Linear System Information=\n', 'Linear System Information::Number of Equations=26996736\n', 'Linear System Information::Number of Nonzero 

TypeError: list.append() takes exactly one argument (5 given)

In [ ]:
# 存储到数据库
df.to_sql('hpcg', conn, if_exists='replace', index=False)

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM hpcg ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

In [ ]:
# 关闭游标和连接
cursor.close()

# 关闭数据库连接
conn.close()